## Context Manager in Python

**Managing Resources**<br>
In any programming language, the usage of resources like file operations or database connections is very common. But these resources are limited in supply. Therefore, the main problem lies in making sure to release these resources after usage. If they are not released then it will lead to resource leakage and may cause the system to either slow down or crash. It would be very helpful if user have a mechanism for the automatic setup and teardown of resources.<br>

- Python provides an easy way to manage resources: Context Managers
- `with` keyword is used for context Managers
- When expresseion after `with` keyword is evaluated,it result in an object that performs context management
- Context managers can be written using classes or functions(with decorators)


#### Creating a Context Managers using Classes
When creating context managers using classes, user need to ensure that the class has the methods:<br>
- `__enter__()`: It returns the resource that has to be managed
- `__exit__()`: It does not return anything but perform clean up activities

In [1]:
class ContextManager(): 
    def __init__(self): 
        print('init method called') 
          
    def __enter__(self): 
        print('enter method called') 
        return self
        
    # the parameters in exit method are used to manage exceptions and has to be present 
    def __exit__(self, exc_type, exc_value, exc_traceback): 
        print('exit method called') 
  
with ContextManager() as manager: 
    print('with statement block') 

init method called
enter method called
with statement block
exit method called


- Execution of `ContextManager()` calls the `__init__()`.
- Then `__enter__()` is called and the object that has to be managed is returned and set to `manager` variable
- Then code within `with` block is executed
- When the code in `with` block ends, the `__exit__()` method is called lastly.

In [2]:
# Python program showing file management using  context manager 
 
class FileManager(): 
    def __init__(self, filename, mode): 
        self.filename = filename 
        self.mode = mode 
        self.file = None
          
    def __enter__(self): 
        self.file = open(self.filename, self.mode) 
        return self.file
      
    def __exit__(self, exc_type, exc_value, exc_traceback): 
        self.file.close() 
  
# loading a file  
with FileManager('file-examples/test_context_managers.txt', 'w') as f: 
    f.write('Testing File Management using Context Managers') 
  
print(f.closed) 

True


In [ ]:
# Python program shows the connection management for MongoDB 
  
from pymongo import MongoClient 
  
class MongoDBConnectionManager(): 
    def __init__(self, hostname, port): 
        self.hostname = hostname 
        self.port = port 
        self.connection = None
  
    def __enter__(self): 
        self.connection = MongoClient(self.hostname, self.port) 
        return self
  
    def __exit__(self, exc_type, exc_value, exc_traceback): 
        self.connection.close() 
  
# connecting with a localhost 
with MongoDBConnectionManager('localhost', 27017) as mongo: 
    collection = mongo.connection.SampleDb.test 
    collection.insert({'_id': 1, 'name':'Saif'}) 
    data = collection.find({'_id': 1})
    print(data.next())

- Execution of `MongoDBConnectionManager()` calls the `__init__()` which set the value for hostname and port.
- Then `__enter__()` is called and the object that has to be managed is returned and set to `mongo` variable
- Then code within `with` block is executed where
    - `mongo.connection` is used to get a connection
    - Then using that connection we open `test` collection within `SampleDB` database.
    - The collection is assigned to `collection` variable.
    - We then query the collection to look for document having `_id:1` which return a cursor object
    - We then use `next()` on cursor to print out the first document in the cursor.
- When the code in `with` block ends, the `__exit__()` method is called lastly that closes the database connection

### Creating Context Managers using `contextmanager` decorator
- **We can simply make any function a context manager with the help of `contextlib.contextmanager` decorator without having to write a separate class or `__enter__()` and `__exit__()` functions.**
- We have to use `contextlib.contextmanager` to decorate a generator function which yields exactly once
- Everything before `yield` is considered to be `__enter__()` section and everything after, to be `__exit__()` section.
- The generator function should yield the resource.

In [3]:
from contextlib import contextmanager 
  
@contextmanager
def ContextManager(): 
      
    # Before yield as the enter method 
    print("Enter method called") 
    yield
      
    # After yield as the exit method 
    print("Exit method called") 
  
with ContextManager() as manager:  
    print('with statement block')  

Enter method called
with statement block
Exit method called


In [4]:
# Python program showing file management using context manager created using decorators

from contextlib import contextmanager

@contextmanager
def file_manager(filename, mode):
    f = open(filename, mode)
    try:
        yield f 
    finally:
        f.close()
 
with file_manager('file-examples/test_context_managers_decorator.txt', 'w') as f: 
    f.write('Testing File Management using Context Managers created using Decorators') 
  
print(f.closed) 

True


- Before `yield` statement, we created file descriptor based on the arguments like we did in `__enter__()`
- Instead of returning the object that has to be managed from `__enter__()`, we used generator that yielded file descriptor.
- This yielded value is set to the variable `f` in expression `with file_manager(filename, mode) as f:`
- Then the code within `with` block is executed
- Just after execution of code in `with` block, the code after `yield` statement is executed


In [ ]:
# tasks.tasksdb_tinydb.py

import tinydb

class TasksDB_TinyDB():  # noqa : E801
    """Wrapper class for TinyDB.

    The methods in this class need to match
    all database interaction classes.

    So far, this is:
    TasksDB_MongoDB found in tasksdb_pymongo.py.
    TasksDB_TinyDB found in tasksdb_tinydb.py.
    """

    def __init__(self, db_path):  # type (str) -> ()
        """Connect to db."""
        self._db = tinydb.TinyDB(db_path + '/tasks_db.json')

    def add(self, task):  # type (dict) -> int
        """Add a task dict to db."""
        task_id = self._db.insert(task)
        task['id'] = task_id
        self._db.update(task, doc_ids=[task_id])
        return task_id

def start_tasks_db(db_path):  # type (str) -> TasksDB_MongoDB object
    """Connect to db."""
    return TasksDB_TinyDB(db_path)

In [ ]:
# tasks.add() within "with" block calls this function

def add(task):  # type: (Task) -> int
    """Add a task (a Task object) to the tasks database."""
    if not isinstance(task, Task):
        raise TypeError('task must be Task object')
    if not isinstance(task.summary, string_types):
        raise ValueError('task.summary must be string')
    if not ((task.owner is None) or
            isinstance(task.owner, string_types)):
        raise ValueError('task.owner must be string or None)')
    # We test for this in ch5, so keep this commented out to let
    # the ch5 test fail.
    #
    # if not isinstance(task.done, bool):
    #     raise ValueError('task.done must be True or False')
    if task.id is not None:
        raise ValueError('task.id must None')
    if _tasksdb is None:
        raise UninitializedDatabase()
    task_id = _tasksdb.add(task._asdict())
    # _taskdb holds the TaskDB_TinyDB object that has add()
    return task_id

In [ ]:
_tasksdb = None

def start_tasks_db(db_path, db_type):  # type: (str, str) -> None
    """Connect API functions to a db."""
    if not isinstance(db_path, string_types):
        raise TypeError('db_path must be a string')
    global _tasksdb
    if db_type == 'tiny':
        import tasks.tasksdb_tinydb
        _tasksdb = tasks.tasksdb_tinydb.start_tasks_db(db_path)
        # _taskdb is a global variable that contain TasksDB_TinyDB object which has _db variable
        # On execution of this line, _taskdb is present globally
    elif db_type == 'mongo':
        import tasks.tasksdb_pymongo
        _tasksdb = tasks.tasksdb_pymongo.start_tasks_db(db_path)
    else:
        raise ValueError("db_type must be a 'tiny' or 'mongo'")

In [ ]:
@contextmanager
def _tasks_db():
    config = tasks.config.get_config() 
    # This call get_config() in tasks.config module which return namedtuple TaskConfig containing keys: db_path and db_type
    tasks.start_tasks_db(config.db_path, config.db_type)
    # This will start the database depending on type of db mongo or tiny and also path
    # After execution of this line, _taskdb is present globally
    # Now code within "with" block will be executed
    yield
    tasks.stop_tasks_db()
    # This will close the connection to database

@tasks_cli.command(help="add a task")
@click.argument('summary')
@click.option('-o', '--owner', default=None,
              help='set the task owner')
def add(summary, owner):
    """Add a task to db."""
    with _tasks_db():
        # Before execution of next line, _taskdb is present globally
        tasks.add(Task(summary, owner))
        # add() returns task_id but we don't need it in CLI
        # Before exiting the "with" block, code after "yield" statement is executed that closes the database